In [2]:
# What are we doing !

# 1. Here we are trying to document the Denovo SNPs from  Thousand Genome samples.
# 2. We take out the SNPs which overlap with STR Denovo regions, to get Unique Denovo SNPs.

# Usage : Unique Denovo SNP locis are used to find out the origin of parental for the STR Denovo locis.

In [ ]:
# we have merged the vcf files in such way that, each region says about variant present in that sample or not.

# here we are focused to see if the 

# Intermediate files for generating Denovo SNPs.

In [68]:
# Packages

import gzip
from collections import Counter
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
import numpy as np
from zipfile import ZipFile
from collections import OrderedDict
import matplotlib.pyplot as plt
import gzip
import re
import ast
import os
import json
import plotly.express as px

In [112]:
# putting the all files  ids into list from vcf header file. ( to further refer with their index to get trio samples)

directory = '../../../../trio_1000_Genomes/SNPs_1KG/'

def VCF_filename(chr_vcf_filename):
    file_ids = []
    
    with gzip.open(f'{directory}{chr_vcf_filename}', 'rt') as vcf:

        for line in vcf:
            line = line.strip().split('\t')

            if line[0].startswith('#CHROM'):
                file_ids.extend(line[9:])
                break
                
    return file_ids        


In [136]:
# list of all sample ids from the vcfs.
All_vcf_files = VCF_filename('1kGP_high_coverage_Illumina.chr1.filtered.SNV_INDEL_SV_phased_panel.vcf.gz')


In [194]:
# putting the all the trio files in dictionary with trio-id and sample index.

directory1 = '../../../../trio_1000_Genomes/'
Trio_info = []

with open(f'{directory1}1KG_Trio_Samples.tsv') as fh:
    fh.readline()

    for files in tqdm(fh):
        samples = {'Ofspng':[],'F':[],'M':[], 'Trio_id':''}
        files = files.strip().split("\t")
        samples['Ofspng'].append(files[2])
        samples['F'].append(files[3])
        samples['M'].append(files[4])
        samples['Trio_id'] = files[0]
        Trio_info.append(samples)


603it [00:00, 236680.27it/s]


In [196]:
# Here we are trying to get the indexs of each samples from the each chromosomes.

# Copy of the dict
Copy_Trio_info = Trio_info.copy()


# iterating the dictionary with 604 trio samples.
for i,trio_dict in enumerate(Copy_Trio_info):
    O = list(trio_dict.values())[0][0]
    F = list(trio_dict.values())[1][0]
    M = list(trio_dict.values())[2][0]

    if O in All_vcf_files and F in All_vcf_files and M in All_vcf_files:  # chr1 : HG02569 HG02567 HG02568

        # lets get the index of the samples in the joint vcf files.
        Copy_Trio_info[i]['Ofspng'].append(All_vcf_files.index(O))
        Copy_Trio_info[i]['F'].append(All_vcf_files.index(F))
        Copy_Trio_info[i]['M'].append(All_vcf_files.index(M))
    
    else: Copy_Trio_info[i]['Ofspng'].append('Failed_Sample')

# save to json file.
with open('../../../Analysis_KT/SNP_Denovo/SNPs_Intermediate_Files/chr1_intermediate_file.json', "w") as outfile:
    json.dump(Copy_Trio_info, outfile)


# clear the dict
Copy_Trio_info.clear()


# SNP & Indel Denovo regions (file creation)

In [197]:
f = open('../../SNP_Denovo/SNPs_Intermediate_Files/chr1_intermediate_file.json')
data = json.loads(f.read())

In [210]:
# function to save files with its name.

def savefile(Region, O_GT, F_GT, M_GT, Trio_id):
    directory2 = '../../../../trio_1000_Genomes/Analysis_KT/SNP_Denovo/'

    file_id = f'{Trio_id}_chr1_SNP_Denovos'
    out = gzip.open(f'{directory2}{file_id}.tsv.gz', 'at')
    
    print(*Region,O_GT, F_GT, M_GT, file =out)
    
    out.close()

In [211]:
# lets analyses the chr1 data and see the speed............


def chr_vcf_file(infile):
    with gzip.open('../../../../trio_1000_Genomes/SNPs_1KG/1kGP_high_coverage_Illumina.chr1.filtered.SNV_INDEL_SV_phased_panel.vcf.gz', 'rt') as vcf:
        for line in tqdm(vcf):
            line = line.strip().split('\t')

            if not line[0].startswith('#'):
                GTs = line[9:]
                Region = line[:9]

                for trio in data:
                    if not trio['Ofspng'][-1] == 'Failed_Sample':
                        O_GT = line[int(trio['Ofspng'][-1])]
                        F_GT = line[int(trio['F'][-1])]
                        M_GT = line[int(trio['M'][-1])]
                        Trio_id = trio['Trio_id']

                        # checking for the Denovo regions.
                        if O_GT != F_GT  and O_GT != M_GT:
                            savefile(Region, O_GT, F_GT, M_GT, Trio_id)



10781it [00:32, 329.55it/s]


KeyboardInterrupt: 

In [212]:
data

[{'Ofspng': ['HG00405', 187],
  'F': ['HG00403', 185],
  'M': ['HG00404', 186],
  'Trio_id': 'TRIO0001'},
 {'Ofspng': ['HG00408', 190],
  'F': ['HG00406', 188],
  'M': ['HG00407', 189],
  'Trio_id': 'TRIO0002'},
 {'Ofspng': ['HG00420', 195],
  'F': ['HG00418', 193],
  'M': ['HG00419', 194],
  'Trio_id': 'TRIO0003'},
 {'Ofspng': ['HG00423', 198],
  'F': ['HG00421', 196],
  'M': ['HG00422', 197],
  'Trio_id': 'TRIO0004'},
 {'Ofspng': ['HG00429', 201],
  'F': ['HG00427', 199],
  'M': ['HG00428', 200],
  'Trio_id': 'TRIO0005'},
 {'Ofspng': ['HG00438', 204],
  'F': ['HG00436', 202],
  'M': ['HG00437', 203],
  'Trio_id': 'TRIO0006'},
 {'Ofspng': ['HG00444', 207],
  'F': ['HG00442', 205],
  'M': ['HG00443', 206],
  'Trio_id': 'TRIO0007'},
 {'Ofspng': ['HG00447', 210],
  'F': ['HG00445', 208],
  'M': ['HG00446', 209],
  'Trio_id': 'TRIO0008'},
 {'Ofspng': ['HG00450', 213],
  'F': ['HG00448', 211],
  'M': ['HG00449', 212],
  'Trio_id': 'TRIO0009'},
 {'Ofspng': ['HG00453', 216],
  'F': ['HG00451

In [ ]:
directory = '../../.'

def chr_vcf_file(infile):
    with gzip.open(f'{directory}{infile}', 'rt') as vcf:
        for line in tqdm(vcf):
            line = line.strip().split('\t')

            if not line[0].startswith('#'):
                
                
                
                
                